# General Relativity and Orbital Precession

This notebook demonstrates general relativistic corrections to orbital mechanics, including perihelion precession and the famous Mercury test.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add src directory to path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

from classical import solve_kepler, orbital_elements_to_state, orbital_period
from relativity import (
    pericenter_precession, 
    pericenter_precession_per_year,
    mercury_precession_test,
    shapiro_delay,
    schwarzschild_radius,
    light_deflection_angle
)
from utils import deg_to_rad, rad_to_deg, AU, solar_mass, G, c

plt.style.use('default')
%matplotlib inline

## Mercury's Perihelion Precession

Mercury's orbit precesses due to several effects:
1. Perturbations from other planets (~531 arcsec/century)
2. General relativistic effects (~43 arcsec/century)
3. Other effects (solar oblateness, etc.)

The GR precession formula is:
$$\Delta\omega = \frac{6\pi GM}{c^2 a(1-e^2)}$$

In [ ]:
# Mercury orbital parameters
a_mercury = 5.791e10  # meters (0.387 AU)
e_mercury = 0.2056
M_sun = solar_mass
mu_sun = G * M_sun

# Calculate Mercury's orbital period
T_mercury = orbital_period(a_mercury, mu_sun)
T_mercury_days = T_mercury / (24 * 3600)

print(f"Mercury's orbital parameters:")
print(f"Semi-major axis: {a_mercury/AU:.3f} AU")
print(f"Eccentricity: {e_mercury:.4f}")
print(f"Orbital period: {T_mercury_days:.1f} days")
print(f"Orbital period: {T_mercury_days/365.25:.2f} years")

In [ ]:
# Calculate GR precession
precession_per_orbit = pericenter_precession(a_mercury, e_mercury, M_sun)
precession_per_year = pericenter_precession_per_year(a_mercury, e_mercury, M_sun, T_mercury)
precession_per_century = precession_per_year * 100

print(f"General Relativistic Precession:")
print(f"Per orbit: {rad_to_deg(precession_per_orbit)*3600:.6f} arcseconds")
print(f"Per year: {precession_per_year:.3f} arcseconds")
print(f"Per century: {precession_per_century:.1f} arcseconds")

# Compare with observations
observed_gr_precession = 42.98  # arcsec/century
error_percentage = abs(precession_per_century - observed_gr_precession) / observed_gr_precession * 100

print(f"\nComparison with observations:")
print(f"Calculated: {precession_per_century:.1f} arcsec/century")
print(f"Observed: {observed_gr_precession:.1f} arcsec/century")
print(f"Relative error: {error_percentage:.2f}%")

## Visualizing Orbital Precession

Let's visualize how the orbit precesses over multiple revolutions.

In [ ]:
def plot_precessing_orbit(a, e, mu, precession_per_orbit, num_orbits=5, num_points_per_orbit=100):
    """Plot multiple orbits showing precession effect"""
    plt.figure(figsize=(12, 12))
    
    colors = plt.cm.viridis(np.linspace(0, 1, num_orbits))
    
    for orbit in range(num_orbits):
        # Calculate accumulated precession
        total_precession = orbit * precession_per_orbit
        
        # Generate orbit points
        M_vals = np.linspace(0, 2*np.pi, num_points_per_orbit)
        x_coords, y_coords = [], []
        
        for M in M_vals:
            pos, vel = orbital_elements_to_state(a, e, M, mu)
            x, y = pos[0], pos[1]
            
            # Apply precession rotation
            cos_prec = np.cos(total_precession)
            sin_prec = np.sin(total_precession)
            x_prec = x * cos_prec - y * sin_prec
            y_prec = x * sin_prec + y * cos_prec
            
            x_coords.append(x_prec / AU)
            y_coords.append(y_prec / AU)
        
        plt.plot(x_coords, y_coords, color=colors[orbit], 
                linewidth=2, label=f'Orbit {orbit+1}', alpha=0.8)
        
        # Mark perihelion
        if orbit == 0 or orbit == num_orbits - 1:
            peri_x = a * (1 - e) * np.cos(total_precession) / AU
            peri_y = a * (1 - e) * np.sin(total_precession) / AU
            marker = 'o' if orbit == 0 else 's'
            label = 'Initial perihelion' if orbit == 0 else 'Final perihelion'
            plt.scatter([peri_x], [peri_y], color=colors[orbit], 
                       s=100, marker=marker, label=label, zorder=5)
    
    # Add the Sun
    plt.scatter([0], [0], color='yellow', s=300, label='Sun', zorder=5, edgecolors='black')
    
    plt.axis('equal')
    plt.xlabel('x (AU)')
    plt.ylabel('y (AU)')
    plt.title(f'Orbital Precession over {num_orbits} Orbits\n'
              f'Total precession: {rad_to_deg(total_precession):.3f}°')
    plt.grid(True, alpha=0.3)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    return total_precession

# For visualization, exaggerate the precession by factor of 10000
exaggerated_precession = precession_per_orbit * 10000
total_prec = plot_precessing_orbit(a_mercury, e_mercury, mu_sun, 
                                  exaggerated_precession, num_orbits=10)
plt.show()

print(f"Note: Precession exaggerated by factor of 10,000 for visualization")
print(f"Actual precession per orbit: {rad_to_deg(precession_per_orbit)*3600:.6f} arcseconds")
print(f"Exaggerated precession shown: {rad_to_deg(exaggerated_precession):.3f} degrees")

## Precession for Different Planets

Let's compare GR precession for different planets in our solar system.

In [ ]:
# Solar system data
planets_data = {
    'Mercury': {'a': 0.387*AU, 'e': 0.2056},
    'Venus':   {'a': 0.723*AU, 'e': 0.0067},
    'Earth':   {'a': 1.000*AU, 'e': 0.0167},
    'Mars':    {'a': 1.524*AU, 'e': 0.0934},
}

print("General Relativistic Precession for Solar System Planets")
print("=" * 60)
print(f"{'Planet':<10} {'Distance (AU)':<15} {'Eccentricity':<15} {'Precession (arcsec/century)':<25}")
print("-" * 60)

planet_names = []
precessions = []

for name, data in planets_data.items():
    a, e = data['a'], data['e']
    T = orbital_period(a, mu_sun)
    precession = pericenter_precession_per_year(a, e, M_sun, T) * 100  # per century
    
    print(f"{name:<10} {a/AU:<15.3f} {e:<15.4f} {precession:<25.3f}")
    
    planet_names.append(name)
    precessions.append(precession)

# Plot comparison
plt.figure(figsize=(10, 6))
bars = plt.bar(planet_names, precessions, color=['gray', 'orange', 'blue', 'red'])
plt.ylabel('GR Precession (arcsec/century)')
plt.title('General Relativistic Precession Comparison')
plt.yscale('log')
plt.grid(True, alpha=0.3)

# Add value labels on bars
for bar, prec in zip(bars, precessions):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height*1.1,
             f'{prec:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## Shapiro Time Delay

The Shapiro delay is the extra time it takes light to travel past a massive object due to gravitational time dilation.

In [ ]:
# Example: radar signal from Earth to Venus and back
r_earth = 1.0 * AU
r_venus = 0.723 * AU

# At superior conjunction (Venus behind Sun)
r_closest_conjunction = solar_radius  # Closest approach to Sun's center

# Calculate delay
delay_conjunction = shapiro_delay(r_earth, r_venus, r_closest_conjunction, M_sun)

print(f"Shapiro Delay Example: Earth-Venus Radar")
print(f"Distance to Earth: {r_earth/AU:.1f} AU")
print(f"Distance to Venus: {r_venus/AU:.1f} AU")
print(f"Closest approach to Sun: {r_closest_conjunction/solar_radius:.1f} solar radii")
print(f"Time delay at superior conjunction: {delay_conjunction*1e6:.1f} microseconds")

# Compare with light travel time
light_travel_time = (r_earth + r_venus) / c
delay_fraction = delay_conjunction / light_travel_time

print(f"\nComparison:")
print(f"Light travel time (Earth-Venus): {light_travel_time:.1f} seconds")
print(f"Shapiro delay: {delay_conjunction*1e6:.1f} microseconds")
print(f"Relative delay: {delay_fraction*1e6:.3f} parts per million")

## Light Deflection by the Sun

Einstein's theory predicts that light passing near the Sun will be deflected by twice the Newtonian prediction.

In [ ]:
# Light deflection at solar limb
impact_parameter = solar_radius
deflection_angle = light_deflection_angle(impact_parameter, M_sun)

# Convert to arcseconds
deflection_arcsec = rad_to_deg(deflection_angle) * 3600

print(f"Light Deflection by the Sun")
print(f"Impact parameter: {impact_parameter/solar_radius:.1f} solar radii")
print(f"Deflection angle: {deflection_arcsec:.2f} arcseconds")

# Compare with observations
observed_deflection = 1.75  # arcseconds, from eclipse measurements
error = abs(deflection_arcsec - observed_deflection) / observed_deflection * 100

print(f"\nComparison with observations:")
print(f"Calculated: {deflection_arcsec:.2f} arcseconds")
print(f"Observed: {observed_deflection:.2f} arcseconds")
print(f"Relative error: {error:.1f}%")

# Plot deflection vs impact parameter
impact_parameters = np.logspace(0, 2, 50) * solar_radius  # 1 to 100 solar radii
deflections = [light_deflection_angle(b, M_sun) for b in impact_parameters]
deflections_arcsec = [rad_to_deg(d) * 3600 for d in deflections]

plt.figure(figsize=(10, 6))
plt.loglog(impact_parameters/solar_radius, deflections_arcsec, 'b-', linewidth=2)
plt.axhline(y=observed_deflection, color='red', linestyle='--', 
           label=f'Observed at solar limb: {observed_deflection} arcsec')
plt.axvline(x=1, color='orange', linestyle='--', alpha=0.7, label='Solar limb')
plt.xlabel('Impact Parameter (Solar Radii)')
plt.ylabel('Deflection Angle (arcseconds)')
plt.title('Light Deflection by the Sun vs Impact Parameter')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

## Schwarzschild Radius and Relativistic Effects

Let's explore the Schwarzschild radius and other relativistic effects for different astronomical objects.

In [ ]:
# Different astronomical objects
objects = {
    'Sun': {'mass': 1.989e30, 'radius': 6.96e8},
    'Earth': {'mass': 5.972e24, 'radius': 6.371e6},
    'Neutron Star': {'mass': 2.8e30, 'radius': 12e3},  # 12 km radius
    'Stellar Black Hole': {'mass': 10 * 1.989e30, 'radius': None},
    'Sgr A* (SMBH)': {'mass': 4.1e6 * 1.989e30, 'radius': None},  # Milky Way central BH
}

print("Schwarzschild Radii and Gravitational Effects")
print("=" * 70)
print(f"{'Object':<20} {'Mass (M_sun)':<15} {'Radius (km)':<15} {'R_s (km)':<15} {'R_s/R':<10}")
print("-" * 70)

for name, data in objects.items():
    mass = data['mass']
    radius = data['radius']
    
    rs = schwarzschild_radius(mass)
    mass_solar = mass / solar_mass
    
    if radius is not None:
        radius_km = radius / 1000
        rs_ratio = rs / radius
        print(f"{name:<20} {mass_solar:<15.2e} {radius_km:<15.1f} {rs/1000:<15.3f} {rs_ratio:<10.2e}")
    else:
        print(f"{name:<20} {mass_solar:<15.2e} {'N/A':<15} {rs/1000:<15.1f} {'N/A':<10}")

# Visualize Schwarzschild radii
names = list(objects.keys())
rs_values = [schwarzschild_radius(objects[name]['mass'])/1000 for name in names]  # in km

plt.figure(figsize=(12, 6))
bars = plt.bar(names, rs_values, color=['orange', 'blue', 'purple', 'black', 'red'])
plt.ylabel('Schwarzschild Radius (km)')
plt.title('Schwarzschild Radii of Different Objects')
plt.yscale('log')
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3)

# Add value labels
for bar, rs in zip(bars, rs_values):
    height = bar.get_height()
    if rs > 1:
        label = f'{rs:.1f} km'
    else:
        label = f'{rs*1000:.1f} m'
    plt.text(bar.get_x() + bar.get_width()/2., height*1.5,
             label, ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## Testing the Mercury Precession Function

Let's use the built-in test function to verify our Mercury calculation.

In [ ]:
# Test Mercury precession calculation
mercury_test = mercury_precession_test()

print("Mercury Perihelion Precession Test")
print("=" * 40)
print(f"Calculated GR precession: {mercury_test['calculated_gr_precession_per_year']:.3f} arcsec/year")
print(f"Observed GR precession: {mercury_test['observed_gr_precession_per_year']:.3f} arcsec/year")
print(f"Relative error: {mercury_test['relative_error']*100:.2f}%")

if mercury_test['relative_error'] < 0.1:  # Less than 10% error
    print("\n✓ Test PASSED: Good agreement with observations!")
else:
    print("\n✗ Test FAILED: Large discrepancy with observations.")

# Summary of key results
print("\n" + "=" * 50)
print("SUMMARY OF GENERAL RELATIVISTIC EFFECTS")
print("=" * 50)
print(f"1. Mercury perihelion precession: {mercury_test['calculated_gr_precession_per_year']*100:.1f} arcsec/century")
print(f"2. Light deflection at solar limb: {deflection_arcsec:.2f} arcseconds")
print(f"3. Shapiro delay (Earth-Venus): {delay_conjunction*1e6:.1f} microseconds")
print(f"4. Sun's Schwarzschild radius: {schwarzschild_radius(M_sun)/1000:.1f} km")
print("\nThese effects, though small, provide crucial tests of Einstein's theory!")